# Data from https://github.com/CSSEGISandData/COVID-19

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
#data
confirmed = "assets/data/time_series_covid19_confirmed_global.csv"
deaths = "assets/data/time_series_covid19_deaths_global.csv"
recovered = "assets/data/time_series_covid19_recovered_global.csv"


In [3]:
# create data frames
confirmed_df = pd.read_csv(confirmed)
deaths_df = pd.read_csv(deaths)
recovered_df = pd.read_csv(recovered)
confirmed_df.head(10)
# tot_locations = len(confirmed_df)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,74,84,94,110,110,120,170,174,237,273
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,123,146,174,186,197,212,223,243,259,277
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,264,302,367,409,454,511,584,716,847,986
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,164,188,224,267,308,334,370,376,390,428
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,3,3,4,4,5,7,7,7,8,8
5,NaN,Antigua and Barbuda,17.0608,-61.7964,0,0,0,0,0,0,...,3,3,7,7,7,7,7,7,7,9
6,NaN,Argentina,-38.4161,-63.6167,0,0,0,0,0,0,...,387,387,502,589,690,745,820,1054,1054,1133
7,NaN,Armenia,40.0691,45.0382,0,0,0,0,0,0,...,249,265,290,329,407,424,482,532,571,663
8,Australian Capital Territory,Australia,-35.4735,149.0124,0,0,0,0,0,0,...,39,39,53,62,71,77,78,80,84,87
9,New South Wales,Australia,-33.8688,151.2093,0,0,0,0,3,4,...,818,1029,1219,1405,1617,1791,2032,2032,2182,2298


In [4]:
# Reset Index to have ID column
confirmed_df.reset_index(inplace=True)
confirmed_df.head()

,index,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,...,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20
0,0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,...,74,84,94,110,110,120,170,174,237,273
1,1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,...,123,146,174,186,197,212,223,243,259,277
2,2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,...,264,302,367,409,454,511,584,716,847,986
3,3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,...,164,188,224,267,308,334,370,376,390,428
4,4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,...,3,3,4,4,5,7,7,7,8,8


In [5]:
# Rename to ID column
confirmed_df = confirmed_df.rename(columns={"index":"id"})
confirmed_df.head()

,id,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,...,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20
0,0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,...,74,84,94,110,110,120,170,174,237,273
1,1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,...,123,146,174,186,197,212,223,243,259,277
2,2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,...,264,302,367,409,454,511,584,716,847,986
3,3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,...,164,188,224,267,308,334,370,376,390,428
4,4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,...,3,3,4,4,5,7,7,7,8,8


In [6]:
#Transpose Columns to get Dates as Row Values
df_melt = confirmed_df.melt(id_vars=['id','Province/State','Country/Region','Lat','Long'])
df_melt.columns = ['id','Province/State','Country/Region','Lat','Long','Date','confirmed_to_date']
df_melt['Date'] = pd.to_datetime(df_melt['Date'])
df_melt = df_melt.sort_values(['Date','id'])
df_melt['Count_Prev_Day'] = df_melt.sort_values('Date').groupby(['id','Province/State','Country/Region',
                                                                 'Lat','Long'])['confirmed_to_date'].shift(1)
df_melt[['Count_Prev_Day']] = df_melt[['Count_Prev_Day']].fillna(value=0)
df_melt['new_confirmed'] = df_melt['confirmed_to_date'] - df_melt['Count_Prev_Day']

df_melt.head()

,id,Province/State,Country/Region,Lat,Long,Date,confirmed_to_date,Count_Prev_Day,new_confirmed
0,0,NaN,Afghanistan,33.0000,65.0000,2020-01-22,0,0.0,0.0
1,1,NaN,Albania,41.1533,20.1683,2020-01-22,0,0.0,0.0
2,2,NaN,Algeria,28.0339,1.6596,2020-01-22,0,0.0,0.0
3,3,NaN,Andorra,42.5063,1.5218,2020-01-22,0,0.0,0.0
4,4,NaN,Angola,-11.2027,17.8739,2020-01-22,0,0.0,0.0


In [7]:
#Check one location to make sure daily count working correctly
df_melt[df_melt['Province/State'] == 'Beijing'].sort_values('Date')

,id,Province/State,Country/Region,Lat,Long,Date,confirmed_to_date,Count_Prev_Day,new_confirmed
50,50,Beijing,China,40.1824,116.4142,2020-01-22,14,0.0,14.0
308,50,Beijing,China,40.1824,116.4142,2020-01-23,22,14.0,8.0
566,50,Beijing,China,40.1824,116.4142,2020-01-24,36,22.0,14.0
824,50,Beijing,China,40.1824,116.4142,2020-01-25,41,36.0,5.0
1082,50,Beijing,China,40.1824,116.4142,2020-01-26,68,41.0,27.0
...,...,...,...,...,...,...,...,...,...
17336,50,Beijing,China,40.1824,116.4142,2020-03-29,577,573.0,4.0
17594,50,Beijing,China,40.1824,116.4142,2020-03-30,577,577.0,0.0
17852,50,Beijing,China,40.1824,116.4142,2020-03-31,580,577.0,3.0
18110,50,Beijing,China,40.1824,116.4142,2020-04-01,580,580.0,0.0


In [8]:
#Drop Count Prev Day column
confirmed_df = df_melt.drop(columns=['Count_Prev_Day'])
confirmed_df.head()

,id,Province/State,Country/Region,Lat,Long,Date,confirmed_to_date,new_confirmed
0,0,NaN,Afghanistan,33.0000,65.0000,2020-01-22,0,0.0
1,1,NaN,Albania,41.1533,20.1683,2020-01-22,0,0.0
2,2,NaN,Algeria,28.0339,1.6596,2020-01-22,0,0.0
3,3,NaN,Andorra,42.5063,1.5218,2020-01-22,0,0.0
4,4,NaN,Angola,-11.2027,17.8739,2020-01-22,0,0.0


In [9]:
#Reset index to have ID column
recovered_df.reset_index(inplace=True)
recovered_df.head()

,index,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,...,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20
0,0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,...,1,2,2,2,2,2,2,5,5,10
1,1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,...,10,17,17,31,31,33,44,52,67,76
2,2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,...,24,65,29,29,31,31,37,46,61,61
3,3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,...,1,1,1,1,1,1,10,10,10,10
4,4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1


In [10]:
#Rename to have id column
recovered_df = recovered_df.rename(columns={"index":"id"})
recovered_df.head()

,id,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,...,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20
0,0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,...,1,2,2,2,2,2,2,5,5,10
1,1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,...,10,17,17,31,31,33,44,52,67,76
2,2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,...,24,65,29,29,31,31,37,46,61,61
3,3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,...,1,1,1,1,1,1,10,10,10,10
4,4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1


In [11]:
#transpose date columns to row values
df_melt = recovered_df.melt(id_vars=['id','Province/State','Country/Region','Lat','Long'])
df_melt.columns = ['id','Province/State','Country/Region','Lat','Long','Date','recovered_to_date']
df_melt['Date'] = pd.to_datetime(df_melt['Date'])
df_melt = df_melt.sort_values(['Date','id'])
df_melt['Count_Prev_Day'] = df_melt.sort_values('Date').groupby(['id','Province/State','Country/Region',
                                                                 'Lat','Long'])['recovered_to_date'].shift(1)
df_melt[['Count_Prev_Day']] = df_melt[['Count_Prev_Day']].fillna(value=0)
df_melt['new_recovered'] = df_melt['recovered_to_date'] - df_melt['Count_Prev_Day']

df_melt.head()

,id,Province/State,Country/Region,Lat,Long,Date,recovered_to_date,Count_Prev_Day,new_recovered
0,0,NaN,Afghanistan,33.0000,65.0000,2020-01-22,0,0.0,0.0
1,1,NaN,Albania,41.1533,20.1683,2020-01-22,0,0.0,0.0
2,2,NaN,Algeria,28.0339,1.6596,2020-01-22,0,0.0,0.0
3,3,NaN,Andorra,42.5063,1.5218,2020-01-22,0,0.0,0.0
4,4,NaN,Angola,-11.2027,17.8739,2020-01-22,0,0.0,0.0


In [12]:
#Filter on one location to make sure working correctly
df_melt[df_melt['Province/State'] == 'Beijing'].sort_values('Date')

,id,Province/State,Country/Region,Lat,Long,Date,recovered_to_date,Count_Prev_Day,new_recovered
41,41,Beijing,China,40.1824,116.4142,2020-01-22,0,0.0,0.0
285,41,Beijing,China,40.1824,116.4142,2020-01-23,0,0.0,0.0
529,41,Beijing,China,40.1824,116.4142,2020-01-24,1,0.0,1.0
773,41,Beijing,China,40.1824,116.4142,2020-01-25,2,1.0,1.0
1017,41,Beijing,China,40.1824,116.4142,2020-01-26,2,2.0,0.0
...,...,...,...,...,...,...,...,...,...
16389,41,Beijing,China,40.1824,116.4142,2020-03-29,412,410.0,2.0
16633,41,Beijing,China,40.1824,116.4142,2020-03-30,415,412.0,3.0
16877,41,Beijing,China,40.1824,116.4142,2020-03-31,418,415.0,3.0
17121,41,Beijing,China,40.1824,116.4142,2020-04-01,418,418.0,0.0


In [13]:
#Drop Prev Day Cout column
recovered_df = df_melt.drop(columns=['Count_Prev_Day'])
recovered_df.head()

,id,Province/State,Country/Region,Lat,Long,Date,recovered_to_date,new_recovered
0,0,NaN,Afghanistan,33.0000,65.0000,2020-01-22,0,0.0
1,1,NaN,Albania,41.1533,20.1683,2020-01-22,0,0.0
2,2,NaN,Algeria,28.0339,1.6596,2020-01-22,0,0.0
3,3,NaN,Andorra,42.5063,1.5218,2020-01-22,0,0.0
4,4,NaN,Angola,-11.2027,17.8739,2020-01-22,0,0.0


In [14]:
#Reset index and save prvious index as id column
deaths_df.reset_index(inplace=True)
deaths_df.head()

,index,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,...,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20
0,0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,...,1,2,4,4,4,4,4,4,4,6
1,1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,...,5,5,6,8,10,10,11,15,15,16
2,2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,...,19,21,25,26,29,31,35,44,58,86
3,3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,...,1,1,3,3,3,6,8,12,14,15
4,4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,...,0,0,0,0,0,2,2,2,2,2


In [15]:
#Rename to have id column
deaths_df = deaths_df.rename(columns={"index":"id"})
deaths_df.head()

,id,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,...,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20
0,0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,...,1,2,4,4,4,4,4,4,4,6
1,1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,...,5,5,6,8,10,10,11,15,15,16
2,2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,...,19,21,25,26,29,31,35,44,58,86
3,3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,...,1,1,3,3,3,6,8,12,14,15
4,4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,...,0,0,0,0,0,2,2,2,2,2


In [16]:
#Transpose date columns to have as row values
df_melt = deaths_df.melt(id_vars=['id','Province/State','Country/Region','Lat','Long'])
df_melt.columns = ['id','Province/State','Country/Region','Lat','Long','Date','deaths_to_date']
df_melt['Date'] = pd.to_datetime(df_melt['Date'])
df_melt = df_melt.sort_values(['Date','id'])
df_melt['Count_Prev_Day'] = df_melt.sort_values('Date').groupby(['id','Province/State','Country/Region',
                                                                 'Lat','Long'])['deaths_to_date'].shift(1)
df_melt[['Count_Prev_Day']] = df_melt[['Count_Prev_Day']].fillna(value=0)
df_melt['new_deaths'] = df_melt['deaths_to_date'] - df_melt['Count_Prev_Day']

df_melt.head()

,id,Province/State,Country/Region,Lat,Long,Date,deaths_to_date,Count_Prev_Day,new_deaths
0,0,NaN,Afghanistan,33.0000,65.0000,2020-01-22,0,0.0,0.0
1,1,NaN,Albania,41.1533,20.1683,2020-01-22,0,0.0,0.0
2,2,NaN,Algeria,28.0339,1.6596,2020-01-22,0,0.0,0.0
3,3,NaN,Andorra,42.5063,1.5218,2020-01-22,0,0.0,0.0
4,4,NaN,Angola,-11.2027,17.8739,2020-01-22,0,0.0,0.0


In [17]:
#Filter on one location to make sure working correctly
df_melt[df_melt['Province/State'] == 'Beijing'].sort_values('Date')

,id,Province/State,Country/Region,Lat,Long,Date,deaths_to_date,Count_Prev_Day,new_deaths
50,50,Beijing,China,40.1824,116.4142,2020-01-22,0,0.0,0.0
308,50,Beijing,China,40.1824,116.4142,2020-01-23,0,0.0,0.0
566,50,Beijing,China,40.1824,116.4142,2020-01-24,0,0.0,0.0
824,50,Beijing,China,40.1824,116.4142,2020-01-25,0,0.0,0.0
1082,50,Beijing,China,40.1824,116.4142,2020-01-26,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
17336,50,Beijing,China,40.1824,116.4142,2020-03-29,8,8.0,0.0
17594,50,Beijing,China,40.1824,116.4142,2020-03-30,8,8.0,0.0
17852,50,Beijing,China,40.1824,116.4142,2020-03-31,8,8.0,0.0
18110,50,Beijing,China,40.1824,116.4142,2020-04-01,8,8.0,0.0


In [18]:
#Drop prev day count column
deaths_df = df_melt.drop(columns=['Count_Prev_Day'])
deaths_df.head()

,id,Province/State,Country/Region,Lat,Long,Date,deaths_to_date,new_deaths
0,0,NaN,Afghanistan,33.0000,65.0000,2020-01-22,0,0.0
1,1,NaN,Albania,41.1533,20.1683,2020-01-22,0,0.0
2,2,NaN,Algeria,28.0339,1.6596,2020-01-22,0,0.0
3,3,NaN,Andorra,42.5063,1.5218,2020-01-22,0,0.0
4,4,NaN,Angola,-11.2027,17.8739,2020-01-22,0,0.0


In [19]:
# Merge confirmed and deaths df on all common columns
semicombined_df = pd.merge(confirmed_df, deaths_df, on=['id','Province/State','Country/Region','Lat','Long', 'Date'])
semicombined_df.head()

,id,Province/State,Country/Region,Lat,Long,Date,confirmed_to_date,new_confirmed,deaths_to_date,new_deaths
0,0,NaN,Afghanistan,33.0000,65.0000,2020-01-22,0,0.0,0,0.0
1,1,NaN,Albania,41.1533,20.1683,2020-01-22,0,0.0,0,0.0
2,2,NaN,Algeria,28.0339,1.6596,2020-01-22,0,0.0,0,0.0
3,3,NaN,Andorra,42.5063,1.5218,2020-01-22,0,0.0,0,0.0
4,4,NaN,Angola,-11.2027,17.8739,2020-01-22,0,0.0,0,0.0


In [20]:
#Merge semimerged df w recovered df on all common columns
combined_df = pd.merge(semicombined_df, recovered_df, on=['id','Province/State','Country/Region','Lat','Long','Date'])
combined_df.head()

,id,Province/State,Country/Region,Lat,Long,Date,confirmed_to_date,new_confirmed,deaths_to_date,new_deaths,recovered_to_date,new_recovered
0,0,NaN,Afghanistan,33.0000,65.0000,2020-01-22,0,0.0,0,0.0,0,0.0
1,1,NaN,Albania,41.1533,20.1683,2020-01-22,0,0.0,0,0.0,0,0.0
2,2,NaN,Algeria,28.0339,1.6596,2020-01-22,0,0.0,0,0.0,0,0.0
3,3,NaN,Andorra,42.5063,1.5218,2020-01-22,0,0.0,0,0.0,0,0.0
4,4,NaN,Angola,-11.2027,17.8739,2020-01-22,0,0.0,0,0.0,0,0.0


In [21]:
#Rename columns to format for SQL
final_df = combined_df.rename(columns={"id":"id_loc","Province/State": "province_state", "Country/Region": "country_region", "Lat":"lat", "Long": "long", "Date": "date"})
final_df.head(10000)

,id_loc,province_state,country_region,lat,long,date,confirmed_to_date,new_confirmed,deaths_to_date,new_deaths,recovered_to_date,new_recovered
0,0,NaN,Afghanistan,33.0000,65.0000,2020-01-22,0,0.0,0,0.0,0,0.0
1,1,NaN,Albania,41.1533,20.1683,2020-01-22,0,0.0,0,0.0,0,0.0
2,2,NaN,Algeria,28.0339,1.6596,2020-01-22,0,0.0,0,0.0,0,0.0
3,3,NaN,Andorra,42.5063,1.5218,2020-01-22,0,0.0,0,0.0,0,0.0
4,4,NaN,Angola,-11.2027,17.8739,2020-01-22,0,0.0,0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3235,226,NaN,Uzbekistan,41.3775,64.5853,2020-04-02,205,205.0,2,2.0,25,25.0
3236,227,NaN,Venezuela,6.4238,-66.5897,2020-04-02,146,146.0,5,5.0,43,43.0
3237,228,NaN,Vietnam,16.0000,108.0000,2020-04-02,233,233.0,0,0.0,75,75.0
3238,229,NaN,Zambia,-15.4167,28.2833,2020-04-02,39,39.0,1,1.0,0,0.0


In [22]:
# conection to SQL database 
rds_connection_string =f"postgres:00Hagase!@localhost:5432/COVID19_DB"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [23]:
engine.table_names()


['covid19']

In [27]:
# table with all data

final_df.to_sql(name='covid19', con=engine, if_exists='replace', index=False)

In [36]:
## Confirm data has been queried properly
covid19_from_SQL=pd.read_sql_query('select * from covid19', con=engine)

covid19_from_SQL.head(10000)

,id_loc,province_state,country_region,lat,long,date,confirmed_to_date,new_confirmed,deaths_to_date,new_deaths,recovered_to_date,new_recovered
0,0,None,Afghanistan,33.0000,65.0000,2020-01-22,0,0.0,0,0.0,0,0.0
1,1,None,Albania,41.1533,20.1683,2020-01-22,0,0.0,0,0.0,0,0.0
2,2,None,Algeria,28.0339,1.6596,2020-01-22,0,0.0,0,0.0,0,0.0
3,3,None,Andorra,42.5063,1.5218,2020-01-22,0,0.0,0,0.0,0,0.0
4,4,None,Angola,-11.2027,17.8739,2020-01-22,0,0.0,0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3235,226,None,Uzbekistan,41.3775,64.5853,2020-04-02,205,205.0,2,2.0,25,25.0
3236,227,None,Venezuela,6.4238,-66.5897,2020-04-02,146,146.0,5,5.0,43,43.0
3237,228,None,Vietnam,16.0000,108.0000,2020-04-02,233,233.0,0,0.0,75,75.0
3238,229,None,Zambia,-15.4167,28.2833,2020-04-02,39,39.0,1,1.0,0,0.0


In [37]:
# covid19_from_SQL=pd.read_sql_query("select * from covid19 WHERE \"country_region\"='Albania'", con=engine)
# # select * from covid19 WHERE "country_region"='Albania'

# covid19_from_SQL.head(10000)

In [39]:
res = covid19_from_SQL.loc[covid19_from_SQL["country_region"] == 'Albania', :]
res


,id_loc,province_state,country_region,lat,long,date,confirmed_to_date,new_confirmed,deaths_to_date,new_deaths,recovered_to_date,new_recovered
1,1,None,Albania,41.1533,20.1683,2020-01-22,0,0.0,0,0.0,0,0.0
46,1,None,Albania,41.1533,20.1683,2020-01-23,0,0.0,0,0.0,0,0.0
91,1,None,Albania,41.1533,20.1683,2020-01-24,0,0.0,0,0.0,0,0.0
136,1,None,Albania,41.1533,20.1683,2020-01-25,0,0.0,0,0.0,0,0.0
181,1,None,Albania,41.1533,20.1683,2020-01-26,0,0.0,0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3016,1,None,Albania,41.1533,20.1683,2020-03-29,212,212.0,10,10.0,33,33.0
3061,1,None,Albania,41.1533,20.1683,2020-03-30,223,223.0,11,11.0,44,44.0
3106,1,None,Albania,41.1533,20.1683,2020-03-31,243,243.0,15,15.0,52,52.0
3151,1,None,Albania,41.1533,20.1683,2020-04-01,259,259.0,15,15.0,67,67.0


In [44]:
covid19_from_SQL.to_csv("covid19.csv", index=False)

In [ ]:
pd.DataFrame.to_json()